In [13]:
import numpy as np
import pandas as pd
import os
from collections import defaultdict

In [23]:
train_path = "D:\Datasets\kaggle_home_credit\csv_files/train/"
optimized_df_path = "D:\Datasets\kaggle_home_credit\csv_files/optimized_df/"
train_dfs = os.listdir(train_path)
print(train_dfs)
# train_dfs = [train_path + i for i in train_dfs]
print(train_dfs)

['train_applprev_1_0.csv', 'train_applprev_1_1.csv', 'train_applprev_2.csv', 'train_base.csv', 'train_credit_bureau_a_1_0.csv', 'train_credit_bureau_a_1_1.csv', 'train_credit_bureau_a_1_2.csv', 'train_credit_bureau_a_1_3.csv', 'train_credit_bureau_a_2_0.csv', 'train_credit_bureau_a_2_1.csv', 'train_credit_bureau_a_2_10.csv', 'train_credit_bureau_a_2_2.csv', 'train_credit_bureau_a_2_3.csv', 'train_credit_bureau_a_2_4.csv', 'train_credit_bureau_a_2_5.csv', 'train_credit_bureau_a_2_6.csv', 'train_credit_bureau_a_2_7.csv', 'train_credit_bureau_a_2_8.csv', 'train_credit_bureau_a_2_9.csv', 'train_credit_bureau_b_1.csv', 'train_credit_bureau_b_2.csv', 'train_debitcard_1.csv', 'train_deposit_1.csv', 'train_other_1.csv', 'train_person_1.csv', 'train_person_2.csv', 'train_static_0_0.csv', 'train_static_0_1.csv', 'train_static_cb_0.csv', 'train_tax_registry_a_1.csv', 'train_tax_registry_b_1.csv', 'train_tax_registry_c_1.csv']
['train_applprev_1_0.csv', 'train_applprev_1_1.csv', 'train_applprev_2.

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
C:\Users\ishtd\AppData\Local\Temp\ipykernel_22628\1888811822.py:1: SyntaxWarning: invalid escape sequence '\D'
  train_path = "D:\Datasets\kaggle_home_credit\csv_files/train/"
C:\Users\ishtd\AppData\Local\Temp\ipykernel_22628\1888811822.py:2: SyntaxWarning: invalid escape sequence '\D'
  optimized_df_path = "D:\Datasets\kaggle_home_credit\csv_files/optimized_df/"


In [16]:
def optimize_dataframe(df):
    """
    Optimize the data types of a pandas DataFrame to reduce memory usage.
    
    Parameters:
    df (pd.DataFrame): The DataFrame to optimize.
    
    Returns:
    pd.DataFrame: A new DataFrame with optimized data types.
    """
    optimized_df = df.copy()
    
    # Optimize numeric columns
    for col in optimized_df.select_dtypes(include=['int', 'float']).columns:
        col_min = optimized_df[col].min()
        col_max = optimized_df[col].max()
        
        if pd.api.types.is_integer_dtype(optimized_df[col]):
            if col_min >= np.iinfo(np.int8).min and col_max <= np.iinfo(np.int8).max:
                optimized_df[col] = optimized_df[col].astype(np.int8)
            elif col_min >= np.iinfo(np.int16).min and col_max <= np.iinfo(np.int16).max:
                optimized_df[col] = optimized_df[col].astype(np.int16)
            elif col_min >= np.iinfo(np.int32).min and col_max <= np.iinfo(np.int32).max:
                optimized_df[col] = optimized_df[col].astype(np.int32)
            else:
                optimized_df[col] = optimized_df[col].astype(np.int64)
        else:
            if col_min >= np.finfo(np.float16).min and col_max <= np.finfo(np.float16).max:
                optimized_df[col] = optimized_df[col].astype(np.float16)
            elif col_min >= np.finfo(np.float32).min and col_max <= np.finfo(np.float32).max:
                optimized_df[col] = optimized_df[col].astype(np.float32)
            else:
                optimized_df[col] = optimized_df[col].astype(np.float64)
    
    # Optimize object columns
    for col in optimized_df.select_dtypes(include=['object']).columns:
        num_unique_values = len(optimized_df[col].unique())
        num_total_values = len(optimized_df[col])
        if num_unique_values / num_total_values < 0.5:
            optimized_df[col] = optimized_df[col].astype('category')
    
    return optimized_df

In [24]:
data_categories = set([file_name.split("_")[1] for file_name in train_dfs])
print(data_categories)
data = defaultdict(list)
for file_name in train_dfs:
    name_split = file_name.split("_")
    data[name_split[1]].append(train_path + file_name)

def optimize_dfs(data_dict,category):
    '''
    Loads data according to the category in a df and stores it in the dictionary

    '''
    for index,path in enumerate(data_dict[category]): 
        print(path)   
        temp_df = pd.read_csv(path)
        optimized_df = optimize_dataframe(temp_df)
        optimized_df.to_pickle(f'{optimized_df_path}/{category}_{index}.pkl')
        data_dict[index] = f'{optimized_df_path}/{category}_{index}.pkl'

def load_data(data_dict,category):
    '''
    Loads data according to the category in a df and stores it in the dictionary

    '''
    df_list = []
    for path in data_dict[category]:    
        temp_df = pd.read_pickle(path)
        df_list.append(temp_df)
    data_dict[category] = df_list


{'applprev', 'debitcard', 'person', 'static', 'deposit', 'other', 'credit', 'tax', 'base.csv'}


In [25]:
optimize_dfs(data,"credit")

D:\Datasets\kaggle_home_credit\csv_files/train/train_credit_bureau_a_1_0.csv


C:\Users\ishtd\AppData\Local\Temp\ipykernel_22628\159905399.py:15: DtypeWarning: Columns (11,12,14,30,44,54) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(path)


D:\Datasets\kaggle_home_credit\csv_files/train/train_credit_bureau_a_1_1.csv
D:\Datasets\kaggle_home_credit\csv_files/train/train_credit_bureau_a_1_2.csv
D:\Datasets\kaggle_home_credit\csv_files/train/train_credit_bureau_a_1_3.csv
D:\Datasets\kaggle_home_credit\csv_files/train/train_credit_bureau_a_2_0.csv
D:\Datasets\kaggle_home_credit\csv_files/train/train_credit_bureau_a_2_1.csv
D:\Datasets\kaggle_home_credit\csv_files/train/train_credit_bureau_a_2_10.csv
D:\Datasets\kaggle_home_credit\csv_files/train/train_credit_bureau_a_2_2.csv
D:\Datasets\kaggle_home_credit\csv_files/train/train_credit_bureau_a_2_3.csv
D:\Datasets\kaggle_home_credit\csv_files/train/train_credit_bureau_a_2_4.csv
D:\Datasets\kaggle_home_credit\csv_files/train/train_credit_bureau_a_2_5.csv


KeyboardInterrupt: 

In [12]:
# base_df contains case id and basic stuff
base_df = pd.read_csv(train_path + "train_base.csv")
# load_data(data,'applprev')
load_data(data,'credit')

C:\Users\ishtd\AppData\Local\Temp\ipykernel_22628\463257710.py:15: DtypeWarning: Columns (11,12,14,30,44,54) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(path)


KeyboardInterrupt: 

In [19]:
# important stats on base df
# total cases = 1526659
#
base_df.head()

,case_id,date_decision,MONTH,WEEK_NUM,target
0,0,2019-01-03,201901,0,0
1,1,2019-01-03,201901,0,0
2,2,2019-01-04,201901,0,0
3,3,2019-01-03,201901,0,0
4,4,2019-01-04,201901,0,1


In [20]:
data['tax'][0].shape


(3275770, 5)

In [21]:
def basic_description(df,nulls_threshold = 0.3):
    print("All columns and their null value counts")
    for i in df.columns:
        nulls = df[i].isna().sum()
        if (nulls/df.shape[0]) < nulls_threshold:
            print(i)
            print("null percentage ", nulls/df.shape[0])
            print("number of unique values ", df[i].nunique())
            print(df[i].unique()[:10])
            print()
        else:
            print(i)
            print(f"{i} has too many nulls")
            print()
    
basic_description(data["credit"][0])


All columns and their null value counts
case_id
null percentage  0.0
number of unique values  335275
[388 405 409 410 411 414 415 416 417 418]

annualeffectiverate_199L
annualeffectiverate_199L has too many nulls

annualeffectiverate_63L
annualeffectiverate_63L has too many nulls

classificationofcontr_13M
null percentage  0.0
number of unique values  11
['ea6782cc' '4408ff0f' 'a55475b1' '01f63ac8' '00135d9c' 'be7b251d'
 '1cf4e481' '2c070815' '0d95a828' '87bdbcba']

classificationofcontr_400M
null percentage  0.0
number of unique values  269
['a55475b1' '42a42e75' '9158339f' 'ec922b98' 'ea6782cc' '01f63ac8'
 '00135d9c' '4d2b9cd0' 'e6e56e83' '601cfe5f']

contractst_545M
null percentage  0.0
number of unique values  43
['7241344e' 'a55475b1' '0dc85f9d' '8f3a197f' 'a52d5641' 'b919198c'
 '3d18d6ef' '885ce291' '7640edc3' 'e2e7d341']

contractst_964M
null percentage  0.0
number of unique values  156
['a55475b1' '7241344e' '4476359f' '9c5b185b' '88c858bd' 'b83056f9'
 '8f3a197f' 'fec76166' 'ec

#####  Interesting things about the data
* Appl prev

    * Important columns : actualdpd_943P - days past due,credacc_credlmt_575A,credamount_590A,credtype_587L,currdebt_94A
    district_544M,downpmt_134A,status_219L,tenor_203L
    * Feature creation column - approvaldate_319D,credtype_587L,district_544M,dtlastpmt_581D,dtlastpmtallstes_3545839D,education_1138M,employedfrom_700D,familystate_726L,isdebitcard_527L,mainoccupationinc_437A,maxdpdtolerance_577P,outstandingdebt_522A,profession_152M,rejectreason_755M,rejectreasonclient_4145042M,revolvingaccount_394A,tenor_203L

    * childnum_21L - why 20 unique values??

* person
    * Important columns : zipcode,education_927M,empl_industry_691L,gender_992L,mainoccupationinc_384A,type_25L

* debitcard
    * Important columns : 
* Tax
    * Important columns : amount_4527230A

    


#### Feature engineering ideas 
* Feature around location (zipcode etc)
* Feature around total debt, acc balance etc,
* counts on number of accts, cards, 
* total of tenor 
* date from features



305137